# Playing with neural networks

In [252]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [253]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.transforms import RandomApply
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
xce = nn.CrossEntropyLoss()

## Setting up a Convolution Neural Network (CNN)

In [254]:
class CNN(nn.Module):
    def __init__(self, in_channels, output_dim):
        super(CNN, self).__init__()

        self.cnn1 = nn.Conv2d(in_channels=in_channels, out_channels=32, kernel_size=5)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        self.cnn2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5)
        self.relu2 = nn.ReLU()

        self.cnn3 = nn.Conv2d(in_channels=in_channels,out_channels=64, kernel_size=5 )
        self.relu3 = nn.ReLU()
        # Adding 2 additional conv layers 
        """         self.cnn3 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5)
        self.relu3 = nn.ReLU()

        self.cnn4 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5)
        self.relu4 = nn.ReLU() """
        # Add more convolutional layers if needed


        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        if in_channels == 1:
            input_dim = 32 * 4 * 4
        else:
            input_dim = 32 * 5 * 5

        self.fc1 = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        out = self.cnn1(x)
        out = self.relu1(out)

        out = self.maxpool1(out)

        out = self.cnn2(out)
        out = self.relu2(out)

        out = self.maxpool2(out)

        out = out.view(out.size(0), -1)

        out = self.fc1(out)

        return out

In [255]:
output_dim = 10
model = CNN(3, output_dim)
print(model)

CNN(
  (cnn1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
  (cnn3): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
  (relu3): ReLU()
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=800, out_features=10, bias=True)
)


In [256]:
learning_rate = 0.1
batch_size = 100
iterations = 2000

In [257]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
transform = transforms.Compose([transforms.ToTensor(),])
#test to see if transform works
""" transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
]) """


' transform = transforms.Compose([\n    transforms.ToTensor(),\n    transforms.RandomHorizontalFlip(),\n    transforms.RandomRotation(10),\n]) '

In [258]:
from itertools import repeat
def inf_loop(data_loader):
    for loader in repeat(data_loader):
        yield from loader

In [259]:
test_mnist_set  = datasets.MNIST(root="./data", train=False, download=True, transform=transform)
train_mnist_set = datasets.MNIST(root="./data", train=True,  download=True, transform=transform)

train_cifar10_set = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test_cifar10_set = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

noisy_transform = transforms.Compose([
    transforms.ToTensor(),
    RandomApply([transforms.GaussianBlur(kernel_size=3)], p=0.5),
    # Add more transformations if needed
])

train_noisy_cifar10_set = datasets.CIFAR10(root="./data", train=True, download=True, transform=noisy_transform)
test_noisy_cifar10_set = datasets.CIFAR10(root="./data", train=False, download=True, transform=noisy_transform)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [265]:

train_cifar10_loader = inf_loop(DataLoader(train_cifar10_set, batch_size = batch_size, shuffle = False))
test_cifar10_loader  =          DataLoader(test_cifar10_set,  batch_size = batch_size, shuffle = False)

train_noisy_cifar10_loader = inf_loop(DataLoader(train_noisy_cifar10_set, batch_size = batch_size, shuffle = False))
test_noisy_cifar10_loader = DataLoader(test_noisy_cifar10_set, batch_size=batch_size, shuffle=False)


In [261]:
loss_list = []
iteration_list = []
accuracy_list = []

model = model.to(device)

for i, (images, labels) in enumerate(train_cifar10_loader, 1):

    images, labels = images.to(device), labels.to(device)

    train  = images

    optimizer.zero_grad()

    outputs = model(train)

    loss = xce(outputs, labels)

    loss.backward()

    optimizer.step()

    if i % 100 == 0:

        # Regular CIFAR-10 evaluation
        correct_regular = 0
        total_regular = 0

        for images, labels in test_cifar10_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            predicted = torch.argmax(outputs, dim=1)
            total_regular += labels.size(0)
            correct_regular += (predicted == labels).sum().item()

        accuracy_regular = 100 * correct_regular / total_regular
        print('Iteration: {:5d}  Loss: {:.4f}  Accuracy regular: {:.2f}%'.format(i, loss.data, accuracy_regular))


        # Noisy CIFAR-10 evaluation
        correct_noisy = 0
        total_noisy = 0

        for images, labels in test_noisy_cifar10_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            predicted = torch.argmax(outputs, dim=1)
            total_noisy += labels.size(0)
            correct_noisy += (predicted == labels).sum().item()

        accuracy_noisy = 100 * correct_noisy / total_noisy
        print('Iteration: {:5d}  Loss: {:.4f}  Accuracy noisy: {:.2f}%'.format(i, loss.data, accuracy_noisy))


    if i == iterations:
        break

Iteration:   100  Loss: 2.1325  Accuracy regular: 25.63%
Iteration:   100  Loss: 2.1325  Accuracy noisy: 25.47%
Iteration:   200  Loss: 1.8787  Accuracy regular: 30.83%
Iteration:   200  Loss: 1.8787  Accuracy noisy: 29.66%
Iteration:   300  Loss: 1.8030  Accuracy regular: 29.81%
Iteration:   300  Loss: 1.8030  Accuracy noisy: 31.11%
Iteration:   400  Loss: 1.6654  Accuracy regular: 40.95%
Iteration:   400  Loss: 1.6654  Accuracy noisy: 40.94%
Iteration:   500  Loss: 1.7237  Accuracy regular: 40.11%
Iteration:   500  Loss: 1.7237  Accuracy noisy: 40.74%
Iteration:   600  Loss: 1.5365  Accuracy regular: 46.66%
Iteration:   600  Loss: 1.5365  Accuracy noisy: 46.26%
Iteration:   700  Loss: 1.4599  Accuracy regular: 49.00%
Iteration:   700  Loss: 1.4599  Accuracy noisy: 48.64%
Iteration:   800  Loss: 1.2653  Accuracy regular: 49.37%
Iteration:   800  Loss: 1.2653  Accuracy noisy: 49.12%
Iteration:   900  Loss: 1.3168  Accuracy regular: 50.80%
Iteration:   900  Loss: 1.3168  Accuracy noisy:

In [266]:
loss_list = []
iteration_list = []
accuracy_list = []


model2 = CNN(3, output_dim)
model2 = model2.to(device)

for i, (images, labels) in enumerate(train_noisy_cifar10_loader, 1):

    images, labels = images.to(device), labels.to(device)

    train  = images

    optimizer.zero_grad()

    outputs = model2(train)

    loss = xce(outputs, labels)

    loss.backward()

    optimizer.step()

    if i % 100 == 0:

        correct = 0
        total = 0

        for images, labels in test_noisy_cifar10_loader:

            images, labels = images.to(device), labels.to(device)

            test = images
            outputs = model2(test)

            # Get predictions from the maximum value
            predicted = torch.max(outputs.data, 1)[1]

            total += len(labels)
            correct += (predicted == labels).sum()

        accuracy = 100 * correct / float(total)

        loss_list.append(loss.data)
        iteration_list.append(i)
        accuracy_list.append(accuracy)
        print('Iteration: {:5d}  Loss: {:.4f}  Accuracy: {:.2f}%'.format(i, loss.data, accuracy))

    if i == iterations:
        break

Iteration:   100  Loss: 2.3110  Accuracy: 10.02%
Iteration:   200  Loss: 2.3017  Accuracy: 10.06%
Iteration:   300  Loss: 2.3097  Accuracy: 10.05%
Iteration:   400  Loss: 2.3068  Accuracy: 10.05%
Iteration:   500  Loss: 2.3191  Accuracy: 10.04%
Iteration:   600  Loss: 2.3106  Accuracy: 10.05%
Iteration:   700  Loss: 2.3018  Accuracy: 10.06%
Iteration:   800  Loss: 2.3093  Accuracy: 10.05%
Iteration:   900  Loss: 2.3074  Accuracy: 10.04%
Iteration:  1000  Loss: 2.3193  Accuracy: 10.05%
Iteration:  1100  Loss: 2.3110  Accuracy: 10.06%
Iteration:  1200  Loss: 2.3019  Accuracy: 10.05%
Iteration:  1300  Loss: 2.3103  Accuracy: 10.05%
Iteration:  1400  Loss: 2.3071  Accuracy: 10.06%
Iteration:  1500  Loss: 2.3196  Accuracy: 10.05%
Iteration:  1600  Loss: 2.3111  Accuracy: 10.04%
Iteration:  1700  Loss: 2.3022  Accuracy: 10.06%
Iteration:  1800  Loss: 2.3098  Accuracy: 10.06%
Iteration:  1900  Loss: 2.3066  Accuracy: 10.04%
Iteration:  2000  Loss: 2.3196  Accuracy: 10.05%
